# End of module project
Part 2 bis - Enrichment

Objective is to compute additional feature to use in simple model

Uses the tennis matches results data between 2020 and 2022 from http://tennis-data.co.uk/alldata.php



In [ ]:
import numpy as np
import pandas as pd

df = pd.read_csv('./2020-2022.csv')
df.head()

In [ ]:
# drop the lines where there are no data in AvgW, AvgL, WRank or LRank
numels = df.shape[0]
df = df.dropna(subset=['AvgW','AvgL','WRank','LRank'],axis=0)
print('Drop NA on AvgW, AvgL, WRank, LRank: ',numels-df.shape[0],' rows dropped')

# drop the lines where AvgW ou AvgL are dubious (more than 50% away from B365W / B365L)
numels = df.shape[0]
df = df.drop(df[df['AvgW']>1.5*df['B365W']].index,axis=0)
df = df.drop(df[df['AvgL']>1.5*df['B365L']].index,axis=0)
print('Drop dubious odds: ',numels-df.shape[0],' rows dropped')

df = df.reset_index(drop=True)

# compute delta rank (difference between winner and loser ranks)
df['DeltaRank'] = df['WRank']-df['LRank']

In [ ]:
# compute new columns
subdf = df.copy()
subdf['W10lastW'] = 0 # number of wins in 10 last matches for winner of match i
subdf['W10lastL'] = 0 # number of wins in 10 last matches for loser of match i
subdf['W5lastW'] = 0 # number of wins in 5 last matches for winner of match i
subdf['W5lastL'] = 0 # number of wins in 5 last matches for loser of match i
for i in range(subdf.shape[0]):
    WW10 = 0 # winner of match i - numbers of wins in last 10 matches
    WL10 = 0 # winner of match i - numbers of losses in last 10 matches
    LW10 = 0 # loser of match i - numbers of wins in last 10 matches
    LL10 = 0 # loser of match i - numbers of losses in last 10 matches
    WW5 = 0 # winner of match i - numbers of wins in last 5 matches
    WL5 = 0 # winner of match i - numbers of losses in last 5 matches
    LW5 = 0 # loser of match i - numbers of wins in last 5 matches
    LL5 = 0 # loser of match i - numbers of losses in last 5 matches
    
    for j in range(i-1):
        if subdf['Winner'][i-j-1]==subdf['Winner'][i] and (WW10+WL10)<10:
            WW10 = WW10+1
        elif subdf['Loser'][i-j-1]==subdf['Winner'][i] and (WW10+WL10)<10:
            WL10 = WL10+1
        
        if subdf['Winner'][i-j-1]==subdf['Loser'][i] and (LW10+LL10)<10:
            LW10 = LW10+1
        elif subdf['Loser'][i-j-1]==subdf['Loser'][i] and (LW10+LL10)<10:
            LL10 = LL10+1
            
        if subdf['Winner'][i-j-1]==subdf['Winner'][i] and (WW5+WL5)<5:
            WW5 = WW5+1
        elif subdf['Loser'][i-j-1]==subdf['Winner'][i] and (WW5+WL5)<5:
            WL5 = WL5+1
        
        if subdf['Winner'][i-j-1]==subdf['Loser'][i] and (LW5+LL5)<5:
            LW5 = LW5+1
        elif subdf['Loser'][i-j-1]==subdf['Loser'][i] and (LW5+LL5)<5:
            LL5 = LL5+1
            
        if (WW+WL)==10 & (LW+LL)==10:
            break           
        
    subdf.loc[i,'W10lastW'] = WW10
    subdf.loc[i,'W5lastW'] = WW5
    subdf.loc[i,'W10lastL'] = LW10
    subdf.loc[i,'W5lastL'] = LW5
    

In [ ]:
# save the new dataframe
subdf.to_csv('./2020-2022-enriched.csv')